In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# 모듈 import

In [7]:
import numpy as np
import pandas as pd

# 데이터 전처리

### 데이터 호출

In [8]:
# 중요 컬럼 지정 데이터 호출
df_cols= ['issue_d',
          'loan_amnt',
          'term',
          'int_rate',
          'grade',
          'sub_grade',
          'revol_util',
          'home_ownership',
          'earliest_cr_line',
          'annual_inc',
          'installment',
          'purpose',
          'dti',
          'loan_status']

df = pd.read_csv('./data/accepted_2007_to_2018Q4.csv', usecols= df_cols)

In [9]:
from IPython.display import display, HTML

display(HTML(df.head(50).to_html()))

,loan_amnt,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,issue_d,loan_status,purpose,dti,earliest_cr_line,revol_util
0,3600.0,36 months,13.99,123.03,C,C4,MORTGAGE,55000.0,Dec-2015,Fully Paid,debt_consolidation,5.91,Aug-2003,29.7
1,24700.0,36 months,11.99,820.28,C,C1,MORTGAGE,65000.0,Dec-2015,Fully Paid,small_business,16.06,Dec-1999,19.2
2,20000.0,60 months,10.78,432.66,B,B4,MORTGAGE,63000.0,Dec-2015,Fully Paid,home_improvement,10.78,Aug-2000,56.2
3,35000.0,60 months,14.85,829.90,C,C5,MORTGAGE,110000.0,Dec-2015,Current,debt_consolidation,17.06,Sep-2008,11.6
4,10400.0,60 months,22.45,289.91,F,F1,MORTGAGE,104433.0,Dec-2015,Fully Paid,major_purchase,25.37,Jun-1998,64.5
5,11950.0,36 months,13.44,405.18,C,C3,RENT,34000.0,Dec-2015,Fully Paid,debt_consolidation,10.20,Oct-1987,68.4
6,20000.0,36 months,9.17,637.58,B,B2,MORTGAGE,180000.0,Dec-2015,Fully Paid,debt_consolidation,14.67,Jun-1990,84.5
7,20000.0,36 months,8.49,631.26,B,B1,MORTGAGE,85000.0,Dec-2015,Fully Paid,major_purchase,17.61,Feb-1999,5.7
8,10000.0,36 months,6.49,306.45,A,A2,RENT,85000.0,Dec-2015,Fully Paid,credit_card,13.07,Apr-2002,34.5
9,8000.0,36 months,11.48,263.74,B,B5,MORTGAGE,42000.0,Dec-2015,Fully Paid,credit_card,34.80,Nov-1994,39.1


In [10]:
'''
| 범주       | Feature                  | 비고            |
| -------- | ------------------------ | ------------- |
| 신용등급     | `grade`, `sub_grade`     | Label/Ordinal |
| 금리       | `int_rate`               | float         |
| 대출/소득 비율 | `loan_amnt / annual_inc` | 파생            |
| 부채비율     | `dti`                    | float         |
| 신용카드 사용률 | `revol_util`             | float         |
| 신용이력길이   | `credit_age`             | 파생            |
| 주거 형태    | `home_ownership`         | OneHot        |
| 대출 목적    | `purpose`                | OneHot        |
| 금리부담 비율  | `int_to_income`          | 파생            |
| 상환비율     | `installment_ratio`      | 파생            |

'''

'\n| 범주       | Feature                  | 비고            |\n| -------- | ------------------------ | ------------- |\n| 신용등급     | `grade`, `sub_grade`     | Label/Ordinal |\n| 금리       | `int_rate`               | float         |\n| 대출/소득 비율 | `loan_amnt / annual_inc` | 파생            |\n| 부채비율     | `dti`                    | float         |\n| 신용카드 사용률 | `revol_util`             | float         |\n| 신용이력길이   | `credit_age`             | 파생            |\n| 주거 형태    | `home_ownership`         | OneHot        |\n| 대출 목적    | `purpose`                | OneHot        |\n| 금리부담 비율  | `int_to_income`          | 파생            |\n| 상환비율     | `installment_ratio`      | 파생            |\n\n'

## 파생변수 생성

In [11]:
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')
df['credit_age'] = (df['issue_d'] - df['earliest_cr_line']).dt.days
df['int_to_income'] = (df['int_rate'] * df['loan_amnt']) / df['annual_inc']
df['installment_ratio'] = df['installment'] / df['loan_amnt']

In [12]:
display(df)

,loan_amnt,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,issue_d,loan_status,purpose,dti,earliest_cr_line,revol_util,credit_age,int_to_income,installment_ratio
0,3600.0,36 months,13.99,123.03,C,C4,MORTGAGE,55000.0,2015-12-01,Fully Paid,debt_consolidation,5.91,2003-08-01,29.7,4505.0,0.915709,0.034175
1,24700.0,36 months,11.99,820.28,C,C1,MORTGAGE,65000.0,2015-12-01,Fully Paid,small_business,16.06,1999-12-01,19.2,5844.0,4.556200,0.033210
2,20000.0,60 months,10.78,432.66,B,B4,MORTGAGE,63000.0,2015-12-01,Fully Paid,home_improvement,10.78,2000-08-01,56.2,5600.0,3.422222,0.021633
3,35000.0,60 months,14.85,829.90,C,C5,MORTGAGE,110000.0,2015-12-01,Current,debt_consolidation,17.06,2008-09-01,11.6,2647.0,4.725000,0.023711
4,10400.0,60 months,22.45,289.91,F,F1,MORTGAGE,104433.0,2015-12-01,Fully Paid,major_purchase,25.37,1998-06-01,64.5,6392.0,2.235692,0.027876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,40000.0,60 months,10.49,859.56,B,B3,MORTGAGE,227000.0,2016-10-01,Current,debt_consolidation,12.75,1995-02-01,64.9,7913.0,1.848458,0.021489
2260697,24000.0,60 months,14.49,564.56,C,C4,RENT,110000.0,2016-10-01,Charged Off,debt_consolidation,18.30,1999-07-01,68.1,6302.0,3.161455,0.023523
2260698,14000.0,60 months,14.49,329.33,C,C4,MORTGAGE,95000.0,2016-10-01,Current,debt_consolidation,23.36,1996-06-01,54.0,7427.0,2.135368,0.023524
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


## term 칼럼 정리

In [13]:
# term 컬럼에서 숫자만 추출하고 정수형으로 변환
df['term'] = df['term'].astype(str).str.extract('(\d+)').astype(float)

## 결측치 처리

In [14]:
df = df.dropna()
# 인덱스를 재설정하여 연속적인 인덱스를 만듭니다.
df = df.reset_index(drop=True)

print(df.shape)
print(df.isnull().sum())

(2257159, 17)
loan_amnt            0
term                 0
int_rate             0
installment          0
grade                0
sub_grade            0
home_ownership       0
annual_inc           0
issue_d              0
loan_status          0
purpose              0
dti                  0
earliest_cr_line     0
revol_util           0
credit_age           0
int_to_income        0
installment_ratio    0
dtype: int64


In [15]:
display(df)
print(df.dtypes)

,loan_amnt,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,issue_d,loan_status,purpose,dti,earliest_cr_line,revol_util,credit_age,int_to_income,installment_ratio
0,3600.0,36.0,13.99,123.03,C,C4,MORTGAGE,55000.0,2015-12-01,Fully Paid,debt_consolidation,5.91,2003-08-01,29.7,4505.0,0.915709,0.034175
1,24700.0,36.0,11.99,820.28,C,C1,MORTGAGE,65000.0,2015-12-01,Fully Paid,small_business,16.06,1999-12-01,19.2,5844.0,4.556200,0.033210
2,20000.0,60.0,10.78,432.66,B,B4,MORTGAGE,63000.0,2015-12-01,Fully Paid,home_improvement,10.78,2000-08-01,56.2,5600.0,3.422222,0.021633
3,35000.0,60.0,14.85,829.90,C,C5,MORTGAGE,110000.0,2015-12-01,Current,debt_consolidation,17.06,2008-09-01,11.6,2647.0,4.725000,0.023711
4,10400.0,60.0,22.45,289.91,F,F1,MORTGAGE,104433.0,2015-12-01,Fully Paid,major_purchase,25.37,1998-06-01,64.5,6392.0,2.235692,0.027876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257154,24000.0,60.0,12.79,543.50,C,C1,MORTGAGE,95000.0,2016-10-01,Current,home_improvement,19.61,1999-12-01,84.4,6149.0,3.231158,0.022646
2257155,24000.0,60.0,10.49,515.74,B,B3,MORTGAGE,108000.0,2016-10-01,Current,debt_consolidation,34.94,1991-02-01,39.0,9374.0,2.331111,0.021489
2257156,40000.0,60.0,10.49,859.56,B,B3,MORTGAGE,227000.0,2016-10-01,Current,debt_consolidation,12.75,1995-02-01,64.9,7913.0,1.848458,0.021489
2257157,24000.0,60.0,14.49,564.56,C,C4,RENT,110000.0,2016-10-01,Charged Off,debt_consolidation,18.30,1999-07-01,68.1,6302.0,3.161455,0.023523


loan_amnt                   float64
term                        float64
int_rate                    float64
installment                 float64
grade                        object
sub_grade                    object
home_ownership               object
annual_inc                  float64
issue_d              datetime64[ns]
loan_status                  object
purpose                      object
dti                         float64
earliest_cr_line     datetime64[ns]
revol_util                  float64
credit_age                  float64
int_to_income               float64
installment_ratio           float64
dtype: object


## 인코딩

In [16]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import pandas as pd
import numpy as np

# ============= 1️⃣ 순서형 인코딩 (grade, sub_grade) =============
# grade: A~G 순서
grade_order = [['A', 'B', 'C', 'D', 'E', 'F', 'G']]
encoder_grade = OrdinalEncoder(categories=grade_order)
df['grade'] = encoder_grade.fit_transform(df[['grade']])

# sub_grade: A1~G5 (문자열 순서로 자동 인코딩)
encoder_subgrade = OrdinalEncoder()
df['sub_grade'] = encoder_subgrade.fit_transform(df[['sub_grade']])

# ============= 2️⃣ 원핫 인코딩 (home_ownership, purpose) =============
encoder_onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot_cols = ['home_ownership', 'purpose']

onehot_encoded = encoder_onehot.fit_transform(df[onehot_cols])
onehot_df = pd.DataFrame(onehot_encoded, columns=encoder_onehot.get_feature_names_out(onehot_cols))

# 원본과 합치기
df = pd.concat([df.drop(columns=onehot_cols), onehot_df], axis=1)

# ============= 3️⃣ 타겟(loan_status) 이진 인코딩 =============
# Fully Paid → 1, 그 외 (Charged Off, Default 등) → 0
df['loan_status'] = df['loan_status'].apply(
    lambda x: 1 if 'Fully Paid' in str(x) else 0
)

print("✅ 인코딩 완료!")
print(df.dtypes.head(15))


✅ 인코딩 완료!
loan_amnt                   float64
term                        float64
int_rate                    float64
installment                 float64
grade                       float64
sub_grade                   float64
annual_inc                  float64
issue_d              datetime64[ns]
loan_status                   int64
dti                         float64
earliest_cr_line     datetime64[ns]
revol_util                  float64
credit_age                  float64
int_to_income               float64
installment_ratio           float64
dtype: object


## TRAIN DATA와 TEST DATA 분리

In [17]:
# 1. 날짜 변환
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

# 2. 분할 기준
split_date = pd.Timestamp('2017-01-01')

# 3. train/test 분리
train_df = df[df['issue_d'] < split_date].copy()  # .copy() 추가
test_df = df[df['issue_d'] >= split_date].copy()   # .copy() 추가

In [18]:
display(train_df)

,loan_amnt,term,int_rate,installment,grade,sub_grade,annual_inc,issue_d,loan_status,dti,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,3600.0,36.0,13.99,123.03,2.0,13.0,55000.0,2015-12-01,1,5.91,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24700.0,36.0,11.99,820.28,2.0,10.0,65000.0,2015-12-01,1,16.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,20000.0,60.0,10.78,432.66,1.0,8.0,63000.0,2015-12-01,1,10.78,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35000.0,60.0,14.85,829.90,2.0,14.0,110000.0,2015-12-01,0,17.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10400.0,60.0,22.45,289.91,5.0,25.0,104433.0,2015-12-01,1,25.37,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257154,24000.0,60.0,12.79,543.50,2.0,10.0,95000.0,2016-10-01,0,19.61,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2257155,24000.0,60.0,10.49,515.74,1.0,7.0,108000.0,2016-10-01,0,34.94,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2257156,40000.0,60.0,10.49,859.56,1.0,7.0,227000.0,2016-10-01,0,12.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2257157,24000.0,60.0,14.49,564.56,2.0,13.0,110000.0,2016-10-01,0,18.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 학습데이터 생성

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

# train_df는 이미 train/test 분리된 학습 데이터입니다. 이 데이터를 사용하여 모델 학습을 준비합니다.

# one-hot 인코딩된 컬럼 이름 정의 (train_lgbm 함수와 동일하게)
onehot_home_ownership_cols = [col for col in train_df.columns if col.startswith('home_ownership_')]
onehot_purpose_cols = [col for col in train_df.columns if col.startswith('purpose_')]

# 특징(features) 목록 정의 (train_lgbm 함수와 동일하게)
features = [
    'loan_amnt', 'term', 'int_rate', 'installment',
    'grade', 'sub_grade',
    'annual_inc',
    'dti', 'revol_util', 'credit_age', 'int_to_income', 'installment_ratio'
] + onehot_home_ownership_cols + onehot_purpose_cols

# 1. 입력/타깃 분리
x_data = train_df[features]
y_data = train_df['loan_status'] # loan_status는 이미 이진 인코딩되어 있습니다.

# 2. 학습/검증 데이터 분리
x_data_train, x_data_valid, y_data_train, y_data_valid = train_test_split(
    x_data, y_data, test_size=0.2, stratify=y_data, random_state=42
)

# 3. 스케일링
scaler = StandardScaler()
x_data_train_norm = scaler.fit_transform(x_data_train)
x_data_valid_norm = scaler.transform(x_data_valid)

# 4. LightGBM용 데이터셋
train_data = lgb.Dataset(x_data_train_norm, label=y_data_train)
valid_data = lgb.Dataset(x_data_valid_norm, label=y_data_valid)

print("✅ 데이터 분리 완료")
print("Train X:", x_data_train.shape)
print("Valid X:", x_data_valid.shape)

✅ 데이터 분리 완료
Train X: (1056812, 32)
Valid X: (264204, 32)


# 모델 생성

In [22]:
from imblearn.combine import SMOTEENN
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, classification_report, log_loss
import numpy as np, seaborn as sns, matplotlib.pyplot as plt, time, sys

# ==============================
# 1️⃣ SMOTE (훈련 세트만 적용)
# ==============================
smote = SMOTEENN(random_state=42)
x_data_train_res, y_data_train_res = smote.fit_resample(x_data_train_norm, y_data_train)
print(f"✅ SMOTE 완료 → {len(x_data_train_res):,} samples")

# ==============================
# 2️⃣ 모델 파라미터 설정
# ==============================
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'device': 'cpu',
    'n_jobs': -1,
    'class_weight': 'balanced',
    'random_state': 42,
    'max_bin': 64,
    'n_estimators': 250,
    'learning_rate': 0.07,
    'num_leaves': 50,
    'max_depth': 6,
    'min_child_samples': 40,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'feature_fraction': 0.9,
    'reg_lambda': 2.0
}

# ==============================
# 3️⃣ 학습 모니터링 함수
# ==============================
def train_with_progress(model, X_train, y_train, X_val, y_val, eval_metric='f1', early_stop=50):
    start_time = time.time()
    best_f1, best_iter = 0, 0
    total_rounds = params['n_estimators']

    print(f"\n🚀 LightGBM Training Started ({total_rounds} rounds)\n")
    sys.stdout.flush()

    for i in range(1, total_rounds + 1):
        # 모델 복제 후 점진적 학습 (iterative 방식)
        model.set_params(n_estimators=i)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='f1',
            verbose=-1
        )

        # 검증 F1 계산
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='macro')

        # 최고 성능 갱신
        if f1 > best_f1:
            best_f1, best_iter = f1, i

        # ETA 계산
        elapsed = time.time() - start_time
        avg_per_iter = elapsed / i
        remaining = (total_rounds - i) * avg_per_iter

        # 콘솔 실시간 출력
        sys.stdout.write(
            f"\r[Progress: {i/total_rounds*100:5.1f}% | "
            f"ETA: {remaining/60:4.1f}m] "
            f"F1={f1:.4f} | Best={best_f1:.4f} (Iter {best_iter})"
        )
        sys.stdout.flush()

    print(f"\n\n✅ Training Complete! Best F1={best_f1:.4f} (Iter={best_iter})")
    print(f"⏱️ Total time: {(time.time()-start_time)/60:.2f} minutes\n")
    return model

# ==============================
# 4️⃣ 모델 학습 (진행률 표시)
# ==============================
model = LGBMClassifier(**params)
best_model = train_with_progress(model, x_data_train_res, y_data_train_res, x_data_valid_norm, y_data_valid)

# ==============================
# 5️⃣ 최종 평가
# ==============================
y_pred = best_model.predict(x_data_valid_norm)
y_proba = best_model.predict_proba(x_data_valid_norm)[:, 1]

print("\n📄 Classification Report:\n")
print(classification_report(y_data_valid, y_pred, digits=4))
print(f"📉 Log Loss: {log_loss(y_data_valid, y_proba):.4f}")


✅ SMOTE 완료 → 699,437 samples

🚀 LightGBM Training Started (250 rounds)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9


/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[Progress:   0.4% | ETA:  3.8m] F1=0.6186 | Best=0.6186 (Iter 1)[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9


/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[Progress:   0.8% | ETA:  3.0m] F1=0.6310 | Best=0.6310 (Iter 2)[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9


...

[Progress:  99.2% | ETA:  0.1m] F1=0.6558 | Best=0.6558 (Iter 248)[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9


/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[Progress:  99.6% | ETA:  0.1m] F1=0.6559 | Best=0.6559 (Iter 249)[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9


/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[Progress: 100.0% | ETA:  0.0m] F1=0.6559 | Best=0.6559 (Iter 249)

✅ Training Complete! Best F1=0.6559 (Iter=249)
⏱️ Total time: 13.19 minutes


📄 Classification Report:

              precision    recall  f1-score   support

           0     0.5084    0.6005    0.5506     84088
           1     0.7963    0.7289    0.7611    180116

    accuracy                         0.6881    264204
   macro avg     0.6523    0.6647    0.6559    264204
weighted avg     0.7047    0.6881    0.6941    264204

📉 Log Loss: 0.6769


In [ ]:
# import joblib
# import os

# # 저장 경로 지정
# save_dir = "/content/drive/MyDrive/[현대로템 K-방산 교육과정 5기]/All Leading Club Loan Data"
# os.makedirs(save_dir, exist_ok=True)

# # 모델 저장
# model_path = os.path.join(save_dir, "best_lgbm_model.pkl")
# joblib.dump(best_model, model_path)
# print(f"✅ 모델이 Google Drive에 저장되었습니다:\n{model_path}")

# # Google Drive에서 모델 불러오기
# loaded_model = joblib.load(model_path)

# # 검증 데이터로 예측 확인
# y_pred_loaded = loaded_model.predict(x_data_valid_norm)
# print("✅ 모델 불러오기 및 예측 성공")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score

# --- DataFrame으로 변환 (feature name 유지)
x_valid_df = pd.DataFrame(x_data_valid_norm, columns=x_data_train.columns)

# --- 배치 단위 예측 함수 ---
def predict_in_batches(model, X, batch_size=20000):
    preds = []
    for i in range(0, X.shape[0], batch_size):
        batch = X.iloc[i:i+batch_size]  # iloc으로 slice
        preds.append(model.predict_proba(batch)[:, 1])
    return np.concatenate(preds)

# --- 예측 (안정적, 빠름) ---
y_proba = predict_in_batches(best_model, x_valid_df, batch_size=20000)

# --- 평가 ---
loss = log_loss(y_data_valid, y_proba)
auc = roc_auc_score(y_data_valid, y_proba)

print(f"📉 Log Loss: {loss:.4f}")
print(f"💡 ROC AUC: {auc:.4f}")
